<a href="https://colab.research.google.com/github/jhagihara/Symptom-Prognosis-Prediction/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
training = pd.read_csv("Symptoms_training.csv")
testing = pd.read_csv("Symptoms_testing.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Symptoms_training.csv'

In [ ]:
np.random.seed(0)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

np.random.seed(0)

class LayerDense:
    def __init__(self, input_size, output_size):
        self.weight = tf.Variable(tf.random.normal([input_size, output_size]), dtype=tf.float32)

        self.bias = tf.Variable(tf.random.normal([output_size]), dtype=tf.float32)

    def forward(self, input):
        input = tf.cast(input, dtype=tf.float32)
        self.output = tf.add(tf.matmul(input, self.weight), self.bias)
        return self.output

class ActivationRelu:
    def forward(self, input):
        self.output = tf.nn.relu(input)
        return self.output

class ActivationSoftmax:
    def forward(self, input):
        exponent = tf.exp(input - tf.reduce_max(input, axis=1, keepdims=True))
        self.output = exponent / tf.reduce_sum(exponent, axis=1, keepdims=True)
        return self.output

class LossCategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            y_true = tf.cast(y_true, dtype=tf.int32)
            y_true_one_hot = tf.one_hot(y_true, depth=y_pred.shape[1])
        else:
            y_true_one_hot = y_true

        correct_confidences = tf.reduce_sum(y_pred_clipped * y_true_one_hot, axis=1)
        negative_log_likelihoods = -tf.math.log(correct_confidences)
        return negative_log_likelihoods

    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = tf.reduce_mean(sample_losses)
        return data_loss

class OptimizerSGD:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate

    def update_parameters(self, trainable_variables, gradients):
        updated_weights = []
        for var, grad in zip(trainable_variables, gradients):
            if isinstance(var, tf.Variable):
                updated_weights.append(var - self.learning_rate * grad)
            else:
                updated_weights.append(var)
        return updated_weights


class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            self.layers.append(LayerDense(layer_sizes[i], layer_sizes[i + 1]))
            stddev = np.sqrt(2 / (layer_sizes[i] + layer_sizes[i + 1]))
            self.layers[-1].weight.assign(tf.random.normal([layer_sizes[i], layer_sizes[i + 1]], stddev=stddev))
        self.activations = [ActivationRelu() for _ in range(len(layer_sizes) - 2)] + [ActivationSoftmax()]
        self.loss = LossCategoricalCrossentropy()
        self.optimizer = OptimizerSGD(learning_rate=0.1)


    def forward(self, x):
        self.activations[0].forward(self.layers[0].forward(x))
        for l in range(1, len(self.layers)):
            self.activations[l].forward(self.layers[l].forward(self.activations[l - 1].output))
        return self.activations[-1].output

    def trainable_variables(self):
      variables = []
      for layer in self.layers:
          variables.append(layer.weight)
          variables.append(layer.bias)
      return variables

    def backward(self, x, y):
        with tf.GradientTape() as tape:
            predictions = self.forward(x)
            loss_value = self.loss.calculate(predictions, y)
        gradients = tape.gradient(loss_value, self.trainable_variables())


        updated_weights = self.optimizer.update_parameters(self.trainable_variables(), gradients)


        for i, layer in enumerate(self.layers):
            layer.weight.assign(updated_weights[2*i])
            layer.bias.assign(updated_weights[2*i + 1])

        return loss_value

    def compute_accuracy(model, X_test, y_test, label_encoder):
      predictions = model.forward(X_test)
      y_pred_encoded = tf.argmax(predictions, axis=1)
      y_pred = label_encoder.inverse_transform(y_pred_encoded.numpy())

      correct_predictions = np.sum(y_pred == y_test)
      total_samples = len(y_test)
      accuracy = correct_predictions / total_samples

      return accuracy


y_train = training['prognosis']
y_test = testing['prognosis']

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

training_copy = training.copy()
testing_copy = testing.copy()
training_copy.drop(['Unnamed: 0', 'prognosis'], axis=1, inplace=True)
testing_copy.drop(['prognosis'], axis=1, inplace=True)

input_size = training_copy.shape[1]
output_size = len(np.unique(y_train_encoded))

model = NeuralNetwork([input_size, 128, 128, 128, 128, 128, 128, output_size])


epochs = 10000
for epoch in range(epochs):
    model.forward  (training_copy)
    loss = model.backward(training_copy, y_train_encoded)
    accuracy = model.compute_accuracy(testing_copy, y_test, label_encoder)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}, Accuracy: {accuracy}")


In [ ]:
testing_copy